# Setting up Event Markers (Triggers) for Your Task

In many EEG or physiological experiments, it’s crucial to **mark** task events using “triggers” (also called **event markers**). By sending numeric codes to your recording system, you can synchronize recorded signals with specific experimental conditions or participant responses.

This notebook illustrates **two main approaches** for sending triggers:
1. **Phasic Triggers**: Short pulses at key moments (e.g., stimulus onset, button press).  
2. **Tonic Triggers**: Holding a trigger line “high” for the duration of an event (then manually resetting).

We’ll use a **Go/No-Go** task example (uploaded to [Pavlovia](https://pavlovia.org/)) to demonstrate how you might embed these triggers into your PsychoPy code. Note that while we showcase **Go/No-Go**, the same principles apply to **any** task requiring event markers.



## 1. General Setup

### 1.1 Hardware and Serial Communication

- **Neurospec MMBT-S** acts as a bridge between your computer’s serial port and the DSI system (or other hardware).  
- On Windows, your serial port might be `COM3`, while on Linux/macOS, it’s often named something like `/dev/ttyUSB0` or `/dev/cu.usbserial`.

**Finding Your COM Port**  
- **Windows**:  
  1. Open **Device Manager** (press Win+X → select “Device Manager”).  
  2. Expand **“Ports (COM & LPT)”**.  
  3. Look for something like **“USB Serial Port (COMX)”**. The `X` in `COMX` is your COM port number.  
- **Linux**:  
  1. Plug in your device.  
  2. Open a terminal and type `dmesg | grep tty` to see which device was assigned.  
  3. Common names are `/dev/ttyUSB0`, `/dev/ttyACM0`, etc.  
- **macOS**:  
  1. Open the **Terminal**.  
  2. Type `ls /dev/tty.*` or `ls /dev/cu.*` to list serial devices.  
  3. Look for something referencing `usbserial` or similar.

Below is an example of how to open the port using Python’s `serial` library. Adjust the port name and settings as needed:


```python
import serial
## Open the serial port; adjust 'COM3' to match your system
Ser = serial.Serial('COM3')
## Immediately reset the trigger line to 0 (idle state)
Ser.write(str.encode(chr(0)))
print("Serial port opened and trigger set to 0.")

### 1.2 Tonic vs. Phasic Triggers: When to Use Which?

After setting up your serial port connection (section 1.1), the next step is to decide **how** you want to send triggers to your EEG/physiology device. Generally, you have **two** main styles:

1. **Tonic (Sustained) Triggers**
2. **Phasic (Transient) Triggers**

Below is a more detailed overview of when each strategy is preferred, with references to common research practices.

#### 1.2.1 Tonic (Sustained) Triggers

**What Is It?**  
- You set the trigger line to a nonzero value at the **start** of an event/condition, keep it high **throughout** that event, and then **manually reset** it to 0 at the end.

**Typical Use Cases**  
1. **Long-Duration Conditions**  
   - For instance, if your experiment has blocks of trials (e.g., 2-minute blocks of task vs. 2-minute blocks of rest) and you want your EEG data to reflect which block is active.  
   - Tonic markers help you see, in the raw data, a continuous “high” value that indicates “Block A” is ongoing versus “Block B.”  

2. **Multi-Step or Complex Events**  
   - In some paradigms, you might have a longer “trial” with sub-events. You might hold a *tonic* trigger for the entire trial, then also send **phasic** pulses within that trial for specific sub-events.  
   - Example: In a working memory task, you raise a tonic trigger for “encoding phase” and only lower it after the final recall response is done.
3. **Neurofeedback or Real-Time Applications**  
   - Some real-time or closed-loop systems need a continuous “state” marker to know if the subject is in a “trial” vs. “rest” phase. A tonic trigger can facilitate that.  

**Pros**  
- **Clear in the data**: You can visually see in the raw EEG/physio signals that “the experiment was in condition X” from time A to time B.  
- **Simplicity**: For truly extended events, it can be simpler to just set once at onset and reset at offset.

**Cons**  
- **Risk of forgetting to reset**: If you leave the trigger high, you might confuse subsequent phases or events.  
- **Less precise** for short or frequent events inside that sustained period.  

**Key Advice**  
- Use tonic triggers when you specifically want a **continuous** marker in the EEG/physio record, or when your block/condition is distinctly separate in time and you do not need to mark sub-events with the same line.  
- If your design also requires marking **shorter** sub-events within the same block, consider **combining** a tonic trigger (for the entire block) with **phasic** triggers (for each sub-event).

#### 1.2.2 Phasic (Transient) Triggers

**What Is It?**  
- You write a nonzero value for only a **brief** period (often 10–50 ms) at the **exact moment** something happens (e.g., stimulus onset, subject response), and then reset to 0 immediately afterward (or automatically after the short duration).

**Typical Use Cases**  
1. **Discrete Stimulus Onsets**  
   - Most common in classic ERP (Event-Related Potential) studies: each stimulus onset is marked by a short pulse so you can epoch the EEG data precisely around that time.  
   - E.g., P300 paradigms often rely on phasic triggers to time-lock the onset of an oddball stimulus.
   

2. **Responses or Rapid Sequences of Events**  
   - Whenever multiple triggers might occur **close together**—for example, if you have a fast-paced task with many short stimuli or responses.  
   - Phasic pulses ensure each event is distinctly marked without overlap.
3. **Short-Lived States**  
   - If you have an event that lasts only a few milliseconds or tens of milliseconds (e.g., a flash or a beep), a phasic trigger can align tightly with that ephemeral event.

**Pros**  
- **High temporal precision**: Perfect for analyzing latencies in ERPs, reaction times, etc.  
- **Minimal risk of overlap**: Because it resets to 0 quickly, you won’t accidentally obscure subsequent triggers.

**Cons**  
- **Not well-suited to representing a sustained state**: You cannot just look at raw data and see “the line is high the whole time.”  
- If your hardware or software requires a certain **minimum** pulse length (e.g., at least 10 ms) to register, you have to ensure your phasic duration is long enough to be reliably detected.

**Key Advice**  
- Use phasic triggers if you want **clean, discrete** event markers for time-locking in your data analysis (ERP or other trial-based analyses).  
- Double-check your hardware’s **minimum pulse width** requirements. Many EEG amplifiers reliably detect pulses as short as 5–10 ms, but some might need 20–50 ms to register.

#### 1.2.3 Additional Literature-Based Insights

- **Clinical/Neuropsychological** tasks (e.g., oddball paradigms, CNV tasks) typically rely on **phasic** pulses for each stimulus or trial event, because analysis focuses on event-locked potentials.  
- **Longer neurofeedback** or **block-based** designs (e.g., fMRI block designs adapted to EEG) might use **tonic** triggers to denote each block’s start/end.  
- **Multi-level** tasks can combine both: a **tonic** marker to indicate which “block” or “task condition” is active, plus **phasic** markers for each stimulus/response within that block.  
- In **rapid-serial-visual-presentation** (RSVP) studies, **phasic** markers are vital for each item in the stream (since items appear quickly), but if you have a top-level block or condition, you might also add a tonic marker for that block.


#### 1.2.4 Deciding Which Method to Use

- **Experiment Duration & Structure**  
  - If your event or condition lasts a significant portion of time (seconds to minutes) and you want your data to reflect that entire period as distinct from the rest, consider a **tonic** marker.  
  - If you have rapid, frequent events where you only need to know **exact** onset times, **phasic** pulses are typically the standard.

- **Analysis Requirements**  
  - For **ERP or time-locked** analyses: short, discrete triggers (phasic) are typically essential.  
  - For identifying **global states** (like “this block is a dual-task condition,” “that block is single-task,” etc.), a tonic approach helps highlight block boundaries.

- **Hardware Considerations**  
  - Some systems need a minimum trigger width (e.g., 5–10 ms). If you go too short, the device might miss it.  
  - If using a tonic trigger, be absolutely sure to reset it—some recording systems can interpret repeated “high” states incorrectly if not managed well.

#### 1.2.5 Conclusion

- **Tonic** = *Sustained*, indicating a condition that persists for a significant duration.  
- **Phasic** = *Short pulses*, precisely time-locked to important, **momentary** events.  
- **Combination** = Use **tonic** for broad phases (like block-level markers) and **phasic** for **specific within-block** triggers (like stimulus onsets or responses).

By reviewing the typical usage in EEG/physiology studies and referencing well-known paradigms (ERP tasks, block-based designs, etc.), you can select the approach—or **mix** of approaches—that best fits your experimental design and analysis goals.



### 1.3 Immediate vs. On-Flip Trigger Timing

Building on the question of **which** trigger style (tonic or phasic) you need (section 1.2), there’s also the matter of **when** you want the trigger to fire in your PsychoPy code. Specifically:

1. **Immediate**: The trigger is sent the moment your code executes that line.  
2. **On-Flip**: The trigger is scheduled to fire precisely at the **next screen refresh** (`win.flip()`).

Why does this matter? In EEG and psychophysics, you often want to **synchronize** your triggers with **visual events**. If your display updates at a certain frame, sending a trigger “immediately” may not perfectly align with that visual onset. Conversely, if the trigger is not tied to any visual event (e.g., it’s only marking an internal logic change or a participant’s key press), an immediate trigger might suffice.

---

#### 1.3.1 Immediate Trigger Calls

**When do they happen?**  
They occur exactly at the line in your code where you call the trigger function—there’s no waiting for a screen refresh.

**Use Case**  
- **Non-visual events** or purely logic-based changes. For instance, sending a trigger to mark the moment a key press is detected, or when your code transitions from one internal state to another.  
- If the event you’re marking doesn’t need to coincide with a **stimulus flip**, an immediate call is straightforward and effective.

**Example**  
```python
# Immediately send a short (phasic) trigger to mark a key press
if 'space' in key_resp.keys:
    send_trigger_phasic(ser, 10, dur=0.01)
```

#### 1.3.2 On-Flip Trigger Calls

**When do they happen?**  
At the **next screen refresh**—the moment `win.flip()` executes in PsychoPy.

**Use Case**  
- **Frame-locked** or **stimulus-locked** events: When you need a trigger to coincide precisely with a new stimulus appearing on screen. This is crucial in EEG studies requiring exact alignment with stimulus onset.

**Example**  
```python
def turn_trigger_on():
    ser.write(str.encode(chr(20)))  # e.g., trigger value 20
    print("Trigger 20 sent at screen flip")
win.callOnFlip(turn_trigger_on)
win.flip()
core.wait(0.05)  # hold for 50 ms if phasic
ser.write(str.encode(chr(0)))  # reset line back to 0
```
Here, the trigger sets to value 20 exactly when the screen buffers swap, meaning the participant sees the new stimulus and the EEG marker changes in the same instant.

**Pros**

- Ideal for precise synchronization of triggers with visual updates (critical for many ERP studies).
- Reduces timing jitter associated with uncertain code execution times.
**Cons**

- Slightly more complex approach (function scheduling + the actual flip).
- If you forget to call win.flip(), the trigger won’t fire at all.

#### 1.3.3 Choosing the Optimal Timing Approach
1. **Immediate**:

- Simpler for marking background logic, user responses, or anything not explicitly tied to the next screen refresh.
- Slightly less precise for visual onset, but sufficient if you don’t need strict frame-locking.
2. **On-Flip**:

- Essential if your stimulus onset marker must match the exact frame the participant sees the stimulus.
- The standard in tasks where accurate time-locking is mandatory (e.g., ERP paradigms).
- Many experiments employ both methods: on-flip triggers for controlling stimulus presentation alignment, and immediate triggers for user responses or internal transitions.



### 1.4 Implementation: Tonic & Phasic Trigger Functions

So far, you’ve learned about **tonic vs. phasic** triggers (section 1.2) and **immediate vs. on-flip** timing (section 1.3). In this final section, we’ll provide **ready-to-use** PsychoPy functions that let you combine these concepts in practice. You can copy and paste these into your experiment script to send event markers via the serial port.

#### Overview of These Functions

1. **Phasic Triggers**  
   - **`send_trigger_phasic`**: Sends a brief (e.g. 10–50 ms) pulse immediately.  
   - **`send_trigger_onflip_phasic`**: Same brief pulse, but scheduled at the next screen flip.  
2. **Tonic Triggers**  
   - **`send_trigger_tonic`**: Sets the line to a nonzero value and **holds** it until manually reset.  
   - **`reset_trigger_tonic`**: Resets a tonic trigger back to 0.  
   - **`send_trigger_onflip`**: Schedules a tonic trigger to begin at the next screen flip.

Together, these functions give you fine-grained control: you can either fire your marker right away or wait until the precise moment the stimulus appears on-screen, and you can choose whether that marker is a short pulse (phasic) or stays “on” until you reset it (tonic).

Below is all the code in one place, with docstrings explaining the parameters and usage.

```python
from psychopy import core

def send_trigger_phasic(ser, data, dur=0.01):
    """
    Send a short trigger pulse by writing `data` to the serial port,
    wait approximately `dur` seconds, then write 0 to turn the trigger off.
    
    Args:
        ser (serial.Serial): An open serial port object.
        data (int): The trigger value to send.
        dur (float): Duration in seconds to keep the trigger high.
    """
    # Mark the start time
    start = core.getTime()
    
    # Write the trigger value
    ser.write(str.encode(chr(data)))
    
    # Measure how long that took
    elapsed = core.getTime() - start
    remaining = dur - elapsed
    
    # Avoid negative waiting if sending took longer than dur
    if remaining > 0:
        core.wait(remaining)
    
    # Reset the trigger to 0
    ser.write(str.encode(chr(0)))

def send_trigger_onflip_phasic(win, ser, trigger_value, dur=0.05):
    """
    Turn a trigger on at the next screen flip, hold it for `dur` seconds,
    then reset to 0. This approach doesn't use core.callLater.
    
    Typically used for precise alignment of a short pulse (phasic) with
    a visual stimulus onset (on the next flip).
    """
    
    def turn_trigger_on():
        ser.write(str.encode(chr(trigger_value)))
        print(f"Trigger {trigger_value} sent")
    
    # 1) Queue the "turn on" for the next flip
    win.callOnFlip(turn_trigger_on)
    
    # 2) Actually flip so that the trigger command executes now
    win.flip()

    # 3) Wait the desired pulse duration
    core.wait(dur)

    # 4) Turn the trigger off
    ser.write(str.encode(chr(0)))
    print("Trigger reset to 0")

def send_trigger_tonic(ser, trigger_value):
    """
    Send a 'tonic' trigger by writing `trigger_value` to the serial port.
    The line remains at this value until you explicitly reset it.
    
    Args:
        ser (serial.Serial): An open serial port object.
        trigger_value (int): The trigger value to send.
    """
    ser.write(str.encode(chr(trigger_value)))
    print(f"[TONIC] Trigger set to {trigger_value} (must reset later).")

def reset_trigger_tonic(ser):
    """
    Manually reset the trigger line to 0 after using a tonic trigger.
    
    Args:
        ser (serial.Serial): An open serial port object.
    """
    ser.write(str.encode(chr(0)))
    print("[TONIC] Trigger reset to 0.")

def send_trigger_onflip(win, ser, trigger_value):
    """
    Turn a 'tonic' trigger on at the next screen flip (i.e., precisely when
    the new frame is drawn). The line remains at this value until you explicitly
    reset it by calling `reset_trigger_tonic`.
    
    Args:
        win (visual.Window): PsychoPy window object.
        ser (serial.Serial): An open serial port object.
        trigger_value (int): The trigger value to send.
    """
    def turn_trigger_on():
        ser.write(str.encode(chr(trigger_value)))
        print(f"[TONIC] Trigger {trigger_value} set on next flip (must reset later).")
    
    # Schedule the trigger command for the next screen refresh
    win.callOnFlip(turn_trigger_on)
    
    # Actually flip to execute the scheduled call
    win.flip()


## 2. Example Task: Go/No-Go on Pavlovia

This section demonstrates how to incorporate triggers (both **phasic** and **tonic**) into a **Go/No-Go** experiment. The same logic applies to any other task; simply adapt the code for your events and conditions. We’ve also uploaded an example task to Pavlonia for the [Phasic](https://gitlab.pavlovia.org/smilingdevil/gonogo_phasic_triggers) and [Tonic]() approach so you can see the implementation in action.


### 2.1 High-Level Task Flow

A typical **Go/No-Go** experiment might follow these steps:

1. **Task Start**  
   - Optionally send a “task start” trigger, either as a brief (phasic) pulse or a tonic signal.  
2. **Trial Begins**  
   - Display either a **Go** or **No-Go** stimulus.  
   - Immediately mark the stimulus onset with a phasic or tonic trigger.  
3. **Participant Responds** (or not)  
   - Collect keypresses.  
   - Determine correctness (Go correct = pressed, No-Go correct = no press).  
   - Send a trigger to mark correct or incorrect responses.  
4. **Trial Ends**  
   - If using **tonic** triggers, reset the line to 0 here (so it doesn’t carry into the next trial).  
5. **Repeat** for the desired number of trials.  
6. **Task End**  
   - Send a “task end” trigger.  
   - Reset the line to 0, and close the serial port.


### 2.2 Adding Triggers in PsychoPy’s Builder

We’ll assume you have a Builder setup with:
- A routine named **“trial”** for each Go/No-Go stimulus presentation.
- A **keyboard component** named `key_resp` for participant responses.
- A **Code Component** in that routine (plus potentially separate routines for “start_trigger” and “goodbye”).

#### 2.2.1 “Begin Experiment” Tab

In your Code Component’s **“Begin Experiment”** section:


In [ ]:
import serial
from psychopy import core, visual, event

# Initialize the serial port for the Neurospec MMBT-S -> DSI
Ser = serial.Serial('COM3')

# Immediately set trigger line to 0 at the experiment start
Ser.write(str.encode(chr(0)))
print("Serial port opened. Trigger line reset to 0.")

# Define trigger codes (integers for each event)
TASK_START   = 10
GO_ONSET     = 1
NOGO_ONSET   = 2
GO_CORRECT   = 3
NOGO_CORRECT = 4
INCORRECT    = 5
TASK_END     = 99

# Define your phasic/tonic trigger functions
def send_trigger(ser, data, dur=0.01):

    start = core.getTime()
    ser.write(str.encode(chr(data)))
    elapsed = core.getTime() - start
    remaining = dur - elapsed
    if remaining > 0:
        core.wait(remaining)
    ser.write(str.encode(chr(0)))


def send_trigger_onflip(win, ser, trigger_value, dur=0.05):

    def turn_trigger_on():
        ser.write(str.encode(chr(trigger_value)))
        print(f"Trigger {trigger_value} sent")
    win.callOnFlip(turn_trigger_on)
    win.flip()
    core.wait(dur)
    ser.write(str.encode(chr(0)))
    print("Trigger reset to 0")

#### 2.2.2 “Begin Routine” Tab (Stimulus Onset)

In the **“Begin Routine”** tab of your `trial` routine’s Code Component, you can decide how to mark stimulus onset. For instance, if your trial dictionary is named `thisTrial` and has a key `'this_image'`:


In [ ]:
# Example: phasic approach for exact onset
if thisTrial['this_image'] == 'go.png':
    send_trigger_phasic(Ser, GO_ONSET, dur=0.01)
elif thisTrial['this_image'] == 'nogo.png':
    send_trigger_phasic(Ser, NOGO_ONSET, dur=0.01)

# If using tonic instead, do:
# if thisTrial['this_image'] == 'go.png':
#     send_trigger_tonic(Ser, GO_ONSET)
# elif thisTrial['this_image'] == 'nogo.png':
#     send_trigger_tonic(Ser, NOGO_ONSET)

#### 2.2.3 “End Routine” Tab (Response Evaluation)
After the participant responds (or doesn’t), you can mark correct vs. incorrect outcomes. For instance:

In [ ]:
# Grab whatever keys were pressed this trial
resp_keys = key_resp.keys  # Might be [], ['space'], or None

# Ensure resp_keys is always a list (some older PsychoPy versions return None)
if resp_keys is None:
    resp_keys = []

# Determine correctness based on image and response
if thisTrial['this_image'] == 'go.png':
    # For a Go trial, correct if the participant pressed 'space'
    if 'space' in resp_keys:
        send_trigger(Ser,GO_CORRECT_TRIGGER)  
        print(f"Correct Go response, trigger {GO_CORRECT_TRIGGER} sent")
    else:
        send_trigger(Ser,INCORRECT_TRIGGER)
        print(f"Incorrect Go response, trigger {INCORRECT_TRIGGER} sent")

elif thisTrial['this_image'] == 'nogo.png':
    # For a NoGo trial, correct if NO key was pressed
    if len(resp_keys) == 0:
        send_trigger(Ser,NOGO_CORRECT_TRIGGER)
        print(f"Correct NoGo response, trigger {NOGO_CORRECT_TRIGGER} sent")
    else:
        send_trigger(Ser,INCORRECT_TRIGGER)
        print(f"Incorrect NoGo response, trigger {INCORRECT_TRIGGER} sent")

else:
    # In case there's some unexpected image name
    print("Warning: Unknown image type, no trigger sent.")


## Building a trigger structure

### The tonic way

### The phasic way

## Adding triggers to your PsychoPy experiment

As an example, lets implement triggers in the [default Go/No-go task](https://gitlab.pavlovia.org/demos/go_nogo) that can be found on Pavlovia

### Open the port at the beginning of your experiment

### Make sure the trigger channel is set to 0 as soon as your experiment starts

### Identify the events that you need to mark, and appropriately call the function to send triggers

### End your experiment
#### Make sure the trigger channel is set back to 0 when your experiment stops
#### Close the port properly